In [10]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import random
from typing import Dict, List, Tuple
import time
import concurrent.futures
from functools import partial
import openai
from tqdm import tqdm
import json

In [ ]:
def generate_feature_label_descriptions(data_dict_df):
    '''
    Using the Data Dictionary, generate feature
    descriptions and label descriptions
    '''
    feature_descriptions = "--- feature description ---\n"
    label_descriptions = "--- label description ---\n"

    # Process all rows except the last one as feature descriptions
    for _, row in data_dict_df.iloc[:-1].iterrows():
        feature_name = row['Feature']
        description = row['Feature Description']
        feature_descriptions += f"{feature_name}: {description}\n"

    # Treat the last row as the label description
    label_row = data_dict_df.iloc[-1]
    label_name = label_row['Feature']
    label_description = label_row['Feature Description']
    label_descriptions += f"{label_name}: {label_description}\n"

    return feature_descriptions + label_descriptions

def initial_prompt_generator(feature_description, target, feature_list, prompt_template):
    '''
    Uses the target and data dictionary to generate an initial prompt
    '''
    if prompt_template == 't-table':
        first_part = f"""You are an expert in the financial sector and banking industry.
Based on the credit information of individuals, please predict the {target}.
I will supply multiple instances with features and the corresponding label for your reference.
Please refer to the table below for detailed descriptions of the features and label:\n"""

        assert feature_list[-1] == target, "Last feature in feature list should be the target"
        description_prompt = generate_feature_label_descriptions(feature_description[feature_description["Feature"].isin(feature_list)])
        prompt = first_part + description_prompt + "--- data ---\n"
    else:
        prompt = """
The following is a table with features in the first few columns and the label in the last column.
I will supply multiple instances with features and the corresponding label for your reference.
"""
    return prompt

def data_to_prompt_generator(training_set, test_data, table_format='t-table'):
    if table_format == "t-table":
        if training_set is not None:
            in_context_data = "\n".join([f"|{'|'.join(map(str, row))}|" for _, row in training_set.iterrows()])
            in_context_examples = "|" + "|".join(test_data.columns) + in_context_data
        else:
            in_context_examples = "|" + "|".join(test_data.columns) + "|"
    elif table_format == "t-anony":
        if training_set is not None:
            in_context_examples = "\n".join([f"|{'|'.join(map(str, row))}|" for _, row in training_set.iterrows()])
        else:
            raise ValueError("Please check the table_format arg provided as it does not exist")

    return in_context_examples

def get_single_prediction(row_data, client, PROMPT_TO_USE, target, table_format, max_tokens=50):
    """Helper function to get prediction for a single row"""
    index, row = row_data  # Correct unpacking to get index and row
    # Check if row is a Series
    if isinstance(row, pd.Series):
        row_values = row.tolist()
    else:
        raise ValueError("Expected row to be a pandas Series")

    # Mask the label in the test row
    row_values[-1] = "<MASK>"
    masked_test_row = f"|{'|'.join(map(str, row_values))}|"

    # Combine prompt with the masked test row
    full_prompt = PROMPT_TO_USE + "\n" + masked_test_row + "\n"
    full_prompt += f"\nPlease use the supplied data to predict the <MASK> {target}. Only respond with 0 or 1.\nAnswer: " if table_format == "t-table" else "\nPlease use the supplied data to predict the <MASK> label. Only respond with 0 or 1.\nAnswer: "

    answer = row[-1]  # Store the ground truth label

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that predicts loan approval outcomes. Only respond with 0 or 1."},
                {"role": "user", "content": full_prompt}
            ],
            max_tokens=max_tokens,
            temperature=0
        )
        
        generated_answer = response.choices[0].message.content.strip()

        if generated_answer in ['0', '1']:
            return int(generated_answer), answer
        else:
            return "invalid", answer

    except Exception as e:
        print(f"Error generating prediction: {e}")
        return "invalid", answer
        
def get_predictions(client, test_data, target, PROMPT_TO_USE, table_format, max_tokens=50, show_template=False, max_workers=4):
    """Modified to use parallel processing"""
    if show_template:
        # Show template for first row only
        first_row = next(test_data.iterrows())
        row_values = first_row[1].tolist()
        row_values[-1] = "<MASK>"
        masked_test_row = f"|{'|'.join(map(str, row_values))}|"
        full_prompt = PROMPT_TO_USE + "\n" + masked_test_row + "\n"
        print("Prompt template:", full_prompt)
        print("Answer template:", first_row[1][-1])

    # Create partial function with fixed arguments
    get_prediction_partial = partial(
        get_single_prediction,
        client=client,
        PROMPT_TO_USE=PROMPT_TO_USE,
        target=target,
        table_format=table_format,
        max_tokens=max_tokens
    )

    y_pred = []
    y_true = []
    non_numeric_count = 0

    #  ThreadPoolExecutor for parallel processing
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        #  all rows for processing
        future_to_row = {executor.submit(get_prediction_partial, row_data): row_data 
                        for row_data in test_data.iterrows()}  # pass the entire row_data tuple
        
        # process results as they complete
        for future in tqdm(concurrent.futures.as_completed(future_to_row), total=len(future_to_row), desc="Processing predictions"):
            prediction, true_value = future.result()
            y_pred.append(prediction)
            y_true.append(true_value)
            
            if prediction == "invalid":
                non_numeric_count += 1
            
            if non_numeric_count > 15:
                break

    print("Num of non-numeric or invalid:", non_numeric_count)
    return y_pred, y_true

def evaluate_in_context_performance(y_pred, y_true):
    filtered_y_true = [yt for yt, yp in zip(y_true, y_pred) if isinstance(yp, int)]
    filtered_y_pred = [yp for yp in y_pred if isinstance(yp, int)]

    y_true = np.array(filtered_y_true)
    y_pred = np.array(filtered_y_pred)
    
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=1)
    recall = recall_score(y_true, y_pred, zero_division=1)
    f1 = f1_score(y_true, y_pred, zero_division=1)

    results = {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }

    return results

def create_in_context_example_sets(df, sample_size, num_sets, target_column, proportions):
    example_sets = {}
    set_counter = 1

    for prop_1 in proportions:
        num_rows_class_1 = int(sample_size * prop_1)
        num_rows_class_0 = sample_size - num_rows_class_1

        for _ in range(num_sets):
            class_1_rows = df[df[target_column] == 1].sample(n=num_rows_class_1, random_state=random.randint(0, 1000))
            class_0_rows = df[df[target_column] == 0].sample(n=num_rows_class_0, random_state=random.randint(0, 1000))

            sampled_df = pd.concat([class_1_rows, class_0_rows]).sample(frac=1, random_state=random.randint(0, 1000)).reset_index(drop=True)
            example_sets[f"Set_{set_counter}_Prop_{prop_1}"] = sampled_df
            set_counter += 1

    return example_sets

def select_top_features(feature_importance_df, top_n):
    return feature_importance_df.nlargest(top_n, 'Importance')['Feature'].tolist()

def run_experiments(feature_importance_df, test_data, client, training_set_df, sample_size, proportions, 
                   num_sets=30, feature_set_sizes=[5, 10, 20, 30, 40], run_multiple=True, table_format="t-table"):
    results = []
    stored_in_context_sets = {}

    for top_n in feature_set_sizes:
        top_features = select_top_features(feature_importance_df, top_n)
        training_set_df_subset = training_set_df[top_features + ['Approved_Flag']]
        test_data_subset = test_data[top_features + ['Approved_Flag']]
        feature_list_for_prompt = test_data_subset.columns.to_list()
        
        print(f"\nStarting experiments for top {top_n} features...")

        intial_prompt_without_data = initial_prompt_generator(
            feature_description,
            "Approved_Flag",
            feature_list_for_prompt,
            table_format
        )

        for prop_1 in proportions:
            print(f"\nTesting with class 1 proportion: {prop_1}")
            in_context_sets = create_in_context_example_sets(
                training_set_df_subset, 
                sample_size, 
                num_sets, 
                'Approved_Flag', 
                [prop_1]
            )

            for set_id, in_context_examples in in_context_sets.items():
                unique_id = f"Features_{top_n}_Prop_{prop_1}_Set_{set_id}"
                stored_in_context_sets[unique_id] = in_context_examples
                print(f"\nEvaluating Set ID: {set_id}")

                times = 4 if run_multiple else 2

                for run_num in range(1, times):
                    data_prompt = data_to_prompt_generator(in_context_examples, test_data_subset, table_format)
                    PROMPT_TO_USE = intial_prompt_without_data + data_prompt

                    y_pred, y_true = get_predictions(
                        client,
                        test_data_subset,
                        'Approved_Flag',
                        PROMPT_TO_USE,
                        table_format,
                        show_template=False
                    )

                    results_dict = evaluate_in_context_performance(y_pred, y_true)

                    run_results = {
                        "Num Features": top_n,
                        "Sample Size": sample_size,
                        "Class 1 Proportion": prop_1,
                        "Set ID": set_id,
                        "Run Number": run_num,
                        "Accuracy": results_dict["Accuracy"],
                        "Precision": results_dict["Precision"],
                        "Recall": results_dict["Recall"],
                        "F1 Score": results_dict["F1 Score"],
                        "Prediction": y_pred
                    }
                    results.append(run_results)
                    print(f"Intermediate results for Run {run_num}: {run_results}")

    results_df = pd.DataFrame(results)
    print("\nAll experiments completed.")
    return results_df, stored_in_context_sets

In [6]:
# Load feature description data globally since it's used in multiple functions
FILE_PATH = 'Indian_bank_data/'
feature_description = pd.read_csv(FILE_PATH+'Capstone - Data Dictionary - Sheet1.csv')

# Load your data
feature_imp_ohe = pd.read_csv(FILE_PATH+'rf_importances.csv')
feature_imp_label = pd.read_csv(FILE_PATH+'rf_importances_label.csv')
feature_description = pd.read_csv(FILE_PATH+'Capstone - Data Dictionary - Sheet1.csv')

train_ohe = pd.read_csv(FILE_PATH+'train_tree.csv')
train_ohe = train_ohe[~train_ohe["Approved_Flag"].isna()]

test_ohe = pd.read_csv(FILE_PATH+'test_tree.csv')
test_ohe = test_ohe[~test_ohe["Approved_Flag"].isna()]

test_ohe_100 = test_ohe.sample(n=100, random_state=42)

In [4]:
# Run t-table experiments
proportions = [0.1, 0.3, 0.5]
# sample_sizes = [16, 32]
sample_sizes = [16, 32]

# Initialize OpenAI client
client = openai.OpenAI(api_key='API-KEY')

for sample_size in sample_sizes:
    experiment_results, stored_in_context_sets = run_experiments(
        feature_imp_ohe, test_ohe_100, client, train_ohe,
        sample_size=sample_size, proportions=proportions,
        feature_set_sizes=[5, 10, 20, 30], num_sets=15,
        run_multiple=False, table_format="t-table"
    )

    file_name = f"experiments_result_t_table_show_{sample_size}.csv"
    experiment_results.to_csv(file_name, index=False)
    print(f"Results for sample_size={sample_size} saved as {file_name}")

/tmp/ipykernel_6833/3130949446.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[-1]  # Store the ground truth label



Starting experiments for top 5 features...

Testing with class 1 proportion: 0.1

Evaluating Set ID: Set_1_Prop_0.1


Processing predictions: 100%|██████████| 100/100 [00:08<00:00, 11.74it/s]
/tmp/ipykernel_6833/3130949446.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[-1]  # Store the ground truth label


Num of non-numeric or invalid: 0
Intermediate results for Run 1: {'Num Features': 5, 'Sample Size': 16, 'Class 1 Proportion': 0.1, 'Set ID': 'Set_1_Prop_0.1', 'Run Number': 1, 'Accuracy': 0.72, 'Precision': 0.2413793103448276, 'Recall': 0.5384615384615384, 'F1 Score': 0.3333333333333333, 'Prediction': [0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]}

Evaluating Set ID: Set_2_Prop_0.1


Processing predictions: 100%|██████████| 100/100 [00:07<00:00, 12.90it/s]
/tmp/ipykernel_6833/3130949446.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[-1]  # Store the ground truth label


...

Num of non-numeric or invalid: 0
Intermediate results for Run 1: {'Num Features': 30, 'Sample Size': 32, 'Class 1 Proportion': 0.5, 'Set ID': 'Set_13_Prop_0.5', 'Run Number': 1, 'Accuracy': 0.13, 'Precision': 0.13, 'Recall': 1.0, 'F1 Score': 0.23008849557522124, 'Prediction': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Evaluating Set ID: Set_14_Prop_0.5


Processing predictions: 100%|██████████| 100/100 [00:10<00:00,  9.23it/s]
/tmp/ipykernel_6833/3130949446.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[-1]  # Store the ground truth label


Num of non-numeric or invalid: 0
Intermediate results for Run 1: {'Num Features': 30, 'Sample Size': 32, 'Class 1 Proportion': 0.5, 'Set ID': 'Set_14_Prop_0.5', 'Run Number': 1, 'Accuracy': 0.13, 'Precision': 0.13, 'Recall': 1.0, 'F1 Score': 0.23008849557522124, 'Prediction': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Evaluating Set ID: Set_15_Prop_0.5


Processing predictions: 100%|██████████| 100/100 [00:10<00:00,  9.10it/s]

Num of non-numeric or invalid: 0
Intermediate results for Run 1: {'Num Features': 30, 'Sample Size': 32, 'Class 1 Proportion': 0.5, 'Set ID': 'Set_15_Prop_0.5', 'Run Number': 1, 'Accuracy': 0.13, 'Precision': 0.13, 'Recall': 1.0, 'F1 Score': 0.23008849557522124, 'Prediction': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

All experiments completed.
Results for sample_size=32 saved as experiments_result_t_table_show_32.csv


In [5]:
#t-anony
proportions = [0.1, 0.3, 0.5]
sample_sizes = [48]

for sample_size in sample_sizes:
    experiment_results, stored_in_context_sets = run_experiments(
        feature_imp_ohe, test_ohe_100, client, train_ohe,
        sample_size=sample_size, proportions=proportions,
        feature_set_sizes=[5, 10, 20, 30, 40], num_sets=5,
        run_multiple=False, table_format="t-anony"
    )

    file_name = f"experiments_result_t_anony_show_{sample_size}.csv"
    experiment_results.to_csv(file_name, index=False)
    print(f"Results for sample_size={sample_size} saved as {file_name}")

/tmp/ipykernel_6833/3130949446.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[-1]  # Store the ground truth label



Starting experiments for top 5 features...

Testing with class 1 proportion: 0.1

Evaluating Set ID: Set_1_Prop_0.1


Processing predictions: 100%|██████████| 100/100 [00:08<00:00, 11.27it/s]
/tmp/ipykernel_6833/3130949446.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[-1]  # Store the ground truth label


Num of non-numeric or invalid: 0
Intermediate results for Run 1: {'Num Features': 5, 'Sample Size': 48, 'Class 1 Proportion': 0.1, 'Set ID': 'Set_1_Prop_0.1', 'Run Number': 1, 'Accuracy': 0.86, 'Precision': 0.3333333333333333, 'Recall': 0.07692307692307693, 'F1 Score': 0.125, 'Prediction': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

Evaluating Set ID: Set_2_Prop_0.1


Processing predictions: 100%|██████████| 100/100 [00:07<00:00, 14.18it/s]
/tmp/ipykernel_6833/3130949446.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[-1]  # Store the ground truth label


...

Num of non-numeric or invalid: 0
Intermediate results for Run 1: {'Num Features': 40, 'Sample Size': 48, 'Class 1 Proportion': 0.5, 'Set ID': 'Set_3_Prop_0.5', 'Run Number': 1, 'Accuracy': 0.13, 'Precision': 0.13, 'Recall': 1.0, 'F1 Score': 0.23008849557522124, 'Prediction': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Evaluating Set ID: Set_4_Prop_0.5


Processing predictions: 100%|██████████| 100/100 [00:13<00:00,  7.40it/s]
/tmp/ipykernel_6833/3130949446.py:86: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  answer = row[-1]  # Store the ground truth label


Num of non-numeric or invalid: 0
Intermediate results for Run 1: {'Num Features': 40, 'Sample Size': 48, 'Class 1 Proportion': 0.5, 'Set ID': 'Set_4_Prop_0.5', 'Run Number': 1, 'Accuracy': 0.13, 'Precision': 0.13, 'Recall': 1.0, 'F1 Score': 0.23008849557522124, 'Prediction': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Evaluating Set ID: Set_5_Prop_0.5


Processing predictions: 100%|██████████| 100/100 [00:09<00:00, 10.38it/s]

Num of non-numeric or invalid: 0
Intermediate results for Run 1: {'Num Features': 40, 'Sample Size': 48, 'Class 1 Proportion': 0.5, 'Set ID': 'Set_5_Prop_0.5', 'Run Number': 1, 'Accuracy': 0.13, 'Precision': 0.13, 'Recall': 1.0, 'F1 Score': 0.23008849557522124, 'Prediction': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

All experiments completed.
Results for sample_size=48 saved as experiments_result_t_anony_show_48.csv


In [ ]:
def create_batch_inference_jsonl(feature_importance_df, test_data, training_set_df, 
                               sample_size, proportions, feature_set_sizes, 
                               output_file_path, table_format="t-table"):
    """Create JSONL file for batch inference with all experimental conditions"""
    inference_data = []
    request_id = 1

    for top_n in feature_set_sizes:
        # Select top features
        top_features = select_top_features(feature_importance_df, top_n)
        training_subset = training_set_df[top_features + ['Approved_Flag']]
        test_subset = test_data[top_features + ['Approved_Flag']]
        feature_list = test_subset.columns.to_list()
        
        initial_prompt = initial_prompt_generator(
            feature_description,
            "Approved_Flag",
            feature_list,
            table_format
        )

        for prop_1 in proportions:
            # Create in-context examples
            in_context_sets = create_in_context_example_sets(
                training_subset,
                sample_size,
                'Approved_Flag',
                [prop_1],
                num_sets=15,
            )

            for set_id, in_context_examples in in_context_sets.items():
                # Generate data prompt
                data_prompt = data_to_prompt_generator(in_context_examples, test_subset, table_format)
                PROMPT_TO_USE = initial_prompt + data_prompt

                # Create entries for each test instance
                for _, row in test_subset.iterrows():
                    row_values = row.tolist()
                    row_values[-1] = "<MASK>"
                    masked_test_row = f"|{'|'.join(map(str, row_values))}|"
                    
                    full_prompt = PROMPT_TO_USE + "\n" + masked_test_row + "\n"
                    full_prompt += f"\nPlease use the supplied data to predict the <MASK> Approved_Flag. Only respond with 0 or 1.\nAnswer:"

                    inference_entry = {
                        "custom_id": f"request-{request_id}-features_{top_n}_prop_{prop_1}_set_{set_id}",
                        "method": "POST",
                        "url": "/v1/chat/completions",
                        "body": {
                            "model": "gpt-4o-mini",
                            "messages": [
                                {
                                    "role": "system",
                                    "content": "You are a helpful assistant that predicts loan approval outcomes. Only respond with 0 or 1."
                                },
                                {
                                    "role": "user",
                                    "content": full_prompt
                                }
                            ],
                            "max_tokens": 50,
                            "temperature": 0
                        }
                    }
                    inference_data.append(inference_entry)
                    request_id += 1

    # Write to JSONL file
    with open(output_file_path, 'w', encoding='utf-8') as f:
        for entry in inference_data:
            f.write(json.dumps(entry, ensure_ascii=False) + '\n')

    print(f"Batch inference data written to {output_file_path}")
    return output_file_path

Batch inference data written to batch_input.jsonl


In [ ]:
# Create batch inference files for different experimental conditions
# T-table 
proportions = [0.1, 0.3, 0.5]
sample_sizes = [48]

for sample_size in sample_sizes:
    output_file = f'batch_inference_t_table_{sample_size}.jsonl'
    jsonl_file = create_batch_inference_jsonl(
        feature_imp_ohe, 
        test_ohe_100, 
        train_ohe,
        sample_size=sample_size, 
        proportions=proportions,
        feature_set_sizes=[5, 10, 20, 30], 
        output_file_path=output_file,
        table_format="t-table"
    )    
    # Submit batch to OpenAI
    batch_input_file = client.files.create(
        file=open(jsonl_file, "rb"),
        purpose="batch"
    )
    
    batch_job = client.batches.create(
        input_file_id=batch_input_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": f"t-table experiment sample_size={sample_size}"
        }
    )
    print(f"Submitted batch job {batch_job.id} for sample_size={sample_size}")

In [12]:
batch_input_file.id

file-PhUZ6mCNDYmQZY3vTLlIhAo2


In [15]:
batch_input_file_id = batch_input_file.id

client.batches.create(
    input_file_id=batch_input_file_id,
    endpoint="/v1/chat/completions",
    completion_window="24h",
    metadata={
      "description": "batch inference"
    }
)

Batch(id='batch_1PcNCbiV3T17vyp2QxC7Pxr0', completion_window='24h', created_at=1726803875, endpoint='/v1/chat/completions', input_file_id='file-PhUZ6mCNDYmQZY3vTLlIhAo2', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1726890275, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'batch inference'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0)


In [21]:
# T-anony 
sample_sizes = [48]

for sample_size in sample_sizes:
    output_file = f'batch_inference_t_anony_{sample_size}.jsonl'
    jsonl_file = create_batch_inference_jsonl(
        feature_imp_ohe, 
        test_ohe_100, 
        train_ohe,
        sample_size=sample_size, 
        proportions=proportions,
        feature_set_sizes=[5, 10, 20, 30, 40], 
        output_file_path=output_file,
        table_format="t-anony"
    )
    
    # Submit batch to OpenAI
    batch_input_file = client.files.create(
        file=open(jsonl_file, "rb"),
        purpose="batch"
    )
    
    batch_job = client.batches.create(
        input_file_id=batch_input_file.id,
        endpoint="/v1/chat/completions",
        completion_window="24h",
        metadata={
            "description": f"t-anony experiment sample_size={sample_size}"
        }
    )
    print(f"Submitted batch job {batch_job.id} for sample_size={sample_size}")

Batch(id='batch_1JSW23j229234mls', completion_window='24h', created_at=1726803875, endpoint='/v1/chat/completions', input_file_id='file-PhUZ6mCNDYmQZY3vTLlIhAo2', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1726890275, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': 'batch inference'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0


In [ ]:
batch_ids=['batch_1PcNCbiV3T17vyp2QxC7Pxr0', 'batch_1JSW23j2s292CSSDFR34mls0' ]
results_dir = './batch_result'

loaded_results = []
for i in range(1, len(batch_ids) + 1):
    result_file_name = f"{results_dir}/batch_{i}_result.jsonl"
    with open(result_file_name, 'r') as file:
        for line in file:
            # Parse the JSON string into a dict and append to the list of loaded results
            json_object = json.loads(line.strip())
            loaded_results.append(json_object)

In [24]:
loaded_results[-1]


{'id': 'batch_req_FL4sHKwXaZhHIHbM9j3rz2wa',
 'custom_id': 'request-646759',
 'response': {'status_code': 200,
  'request_id': '20aa07c101c87d937a770ad035f4d102',
  'body': {'id': 'chatcmpl-9xD2OLUNWq0kp7TjE1e6LilmPopBm',
   'object': 'chat.completion',
   'created': 1723898496,
   'model': 'gpt-4o-mini-2024-07-18',
   'choices': [{'index': 0,
     'message': {'role': 'assistant',
      'content': "[{'role': 'user', 'content': 'Num Features,Sample Size,Class 1 Proportion,Set ID,Run Number,Accuracy,Precision,Recall,F1 Score,Prediction,PR_AUC,ROC_AUC
5,32,0.1,Set_1_Prop_0.1,1,0.54,0.1489361702127659,0.5384615384615384,0.2333333333333333,"[1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0]",0.1209165302782324,0.45092838196286467
",
      'refu